# Предисловие

- Задание ДЗ3: выгрузить данные и сформировать из них dataframe в pandas
- Задание ДЗ3* (доп): кто хочет продвигаться в анализе данных, советую сделать анализ выгруженных данных с помощью pandas

![drakkar image](https://cs8.pikabu.ru/post_img/big/2016/06/15/6/146598388513055440.png)

это третье домашнее задание (второе дз было на повторение линукса - самостоятельно, я не буду его проверять)


## Задание №1

Скачайте файлы **Version 1** Steam data from [Recommender Systems Datasets](http://cseweb.ucsd.edu/~jmcauley/datasets.html#steam_data). 

нам нужны файлы:
- Version 1: Review Data 
- Version 1: User and Item Data 

Ваша задача развернуть данные в pd.DataFrame (ДЗ3)

In [2]:
# Your code should be written below:
import pandas as pd
import ast
import json

In [4]:
with open('australian_user_reviews.json', encoding="utf8") as f:
    row_data = f.readlines()
    new_data = []
    for line in row_data:
        new_data.append(ast.literal_eval(line))

In [5]:
data = pd.DataFrame(new_data)
data.head()

,reviews,user_id,user_url
0,"[{'funny': '', 'posted': 'Posted November 5, 2...",76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"[{'funny': '', 'posted': 'Posted June 24, 2014...",js41637,http://steamcommunity.com/id/js41637
2,"[{'funny': '', 'posted': 'Posted February 3.',...",evcentric,http://steamcommunity.com/id/evcentric
3,"[{'funny': '', 'posted': 'Posted October 14, 2...",doctr,http://steamcommunity.com/id/doctr
4,"[{'funny': '3 people found this review funny',...",maplemage,http://steamcommunity.com/id/maplemage


Для reviews стоит немного предобработать данные, код ниже (следующие 2 блока кода возможно вам не понадобится, всё будет зависеть от того, как вы развернете данные - смотрите сами крч)

In [135]:
# drop duplicated user IDs
data = data.drop_duplicates(subset='user_id')
# drop users with no reviews
data = data[data['reviews'].apply(len) > 0]

In [7]:
def unpack_user_reviews(row):
    "Unpack items from the items in a user's row"
    # items is a series of lists - chain will make one long iterable
    # convert this to a data frame
    rdf = pd.DataFrame.from_records(row.reviews)
    return rdf

game_reviews = pd.concat((unpack_user_reviews(row) for row in data.itertuples()),
                         ignore_index=True)

In [123]:
game_reviews.head()

,funny,helpful,item_id,last_edited,posted,recommend,review
0,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


In [102]:
# Your code should be written below

with open('australian_users_items.json', encoding="utf8") as file:
    row_data_items = file.readlines()
    new_data_items = []
    for line in row_data_items:
        new_data_items.append(ast.literal_eval(line))

In [142]:
items_df = pd.DataFrame(new_data_items)

In [141]:
items_df = items_df.drop_duplicates(subset='user_id')
items_df = items_df[items_df['items'].apply(len) > 0]
items_df.head()

,items,items_count,steam_id,user_id,user_url
0,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888,76561198035864385,js41637,http://steamcommunity.com/id/js41637
2,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137,76561198007712555,evcentric,http://steamcommunity.com/id/evcentric
3,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328,76561197963445855,Riot-Punch,http://steamcommunity.com/id/Riot-Punch
4,"[{'item_id': '300', 'item_name': 'Day of Defea...",541,76561198002099482,doctr,http://steamcommunity.com/id/doctr


In [137]:
def unpack_user_items(row):
    "Unpack items from the items in a user's row"
    # items is a series of lists - chain will make one long iterable
    # convert this to a data frame
    rdf = pd.DataFrame.from_records(row.items)
    return rdf

items = pd.concat((unpack_user_items(row) for row in items_df.itertuples()),
                         ignore_index=False)

In [138]:
items.head()

,item_id,item_name,playtime_2weeks,playtime_forever
0,10,Counter-Strike,0,6
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,0,7
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0


## Подсчитайте следующие статистики: ДЗ3*
### Q1: Количество уникальных юзеров:

In [9]:
# Your code should be written below:
len(pd.Series(data['user_id']).unique())

25457

### Q2: Количество уникального контента:

In [66]:
# Your code should be written below

len(pd.Series(game_reviews['review']).unique())

55312

### Q3: Найдите самый смешной review, рекомендовали ли там игру?


In [134]:
# Your code should be written below

game_reviews_ = game_reviews[game_reviews['funny'].apply(len) > 0]
dict_funny = game_reviews_['funny'].to_dict()
digits = {}
for i in dict_funny.keys():
    digits.update({i: int(''.join(x for x in dict_funny[i] if x.isdigit()))})
for k, v in digits.items():
    if v == max(digits.values()):
        funniest_indx = k
print('Самый смешной review: \'{}\''.format(game_reviews.loc[funniest_indx, 'review']))
if game_reviews.loc[funniest_indx, 'recommend']:
    print('Да, игру рекомендовали.')
else:
    print('Нет, игру не рекомендовали')

Самый смешной review: 'When a player quits EVE and goes to WoW, the average IQ in both games increases.'
Да, игру рекомендовали.


In [65]:
game_reviews.loc[funniest_indx]

funny                       2,598 people found this review funny
helpful        2,200 of 2,593 people (85%) found this review ...
item_id                                                     8500
last_edited                                                     
posted                                 Posted November 20, 2015.
recommend                                                   True
review         When a player quits EVE and goes to WoW, the a...
Name: 21895, dtype: object

### Q4: Уникальные пользователи:
- Сколько пользователей купили все игры;
- Сколько пользователей рекомендовали(recommend:True) все игры.

In [ ]:
# Your code should be written below


### Q5: Посмотрим на активность пользователей: 
#### Постройте следующие гистограммы: 
- Кол-во приобретенных игр на юзера;
- Кол-во оставленных "review" на юзера.

In [ ]:
# Your code should be written below


## Важно!
#### Всё задание можно выполнить за 45 минут  (но это не точно) с учетом прогрузки данных! Если Ваш код работает дольше, то Вы что-то делаете не правильно.  
### Удачи.